In [3]:
import pandas as pd

In [4]:
computational_df= pd.read_csv('comp.csv')
computational_df.head()

,formula,Property band_gap,Property band_gap units,Property SMILES
0,GdCoO$_{3}$,0.1754,eV/atom,NaN
1,PrPt,0.0000,eV/atom,NaN
2,Tl$_{2}$SeO$_{4}$,3.3122,eV/atom,NaN
3,Ba$_{5}$(InSb$_{3}$)$_{2}$,0.0000,eV/atom,NaN
4,NiP$_{2}$,0.5442,eV/atom,NaN


In [5]:
computational_df.rename(columns={"formula":"chemical_formula","Property band_gap":"band_gap"},inplace=True,errors="raise")

In [6]:
computational_df['chemical_formula']=computational_df['chemical_formula'].astype('str').apply(lambda x:x.translate
                                          ({ord(c): None for c in "_${}"}))

In [7]:
from matminer.featurizers.conversions import StrToComposition
computational_df = StrToComposition().featurize_dataframe(computational_df, 'chemical_formula')

In [12]:
from matminer.featurizers.composition import ElementProperty

ep_feat = ElementProperty.from_preset(preset_name="magpie")
computational_df = ep_feat.featurize_dataframe(computational_df, col_id="composition",ignore_errors=True,return_errors=True)  # input the "composition" column to the featurizer
computational_df.head()

/home/kyaw/anaconda3/envs/GNNenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.neighbors.unsupervised module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


,chemical_formula,band_gap,Property band_gap units,Property SMILES,composition,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,MagpieData avg_dev Number,...,MagpieData mean GSmagmom,MagpieData avg_dev GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,ElementProperty Exceptions
0,GdCoO3,0.1754,eV/atom,NaN,"(Gd, Co, O)",8.0,64.0,56.0,23.000000,18.000000,...,0.309694,0.495511,0.0,12.0,194.0,182.0,84.800000,87.360000,12.0,NaN
1,PrPt,0.0000,eV/atom,NaN,"(Pr, Pt)",59.0,78.0,19.0,68.500000,9.500000,...,0.000000,0.000000,0.0,194.0,225.0,31.0,209.500000,15.500000,194.0,NaN
2,Tl2SeO4,3.3122,eV/atom,NaN,"(Tl, Se, O)",8.0,81.0,73.0,32.571429,28.081633,...,0.000000,0.000000,0.0,12.0,194.0,182.0,64.285714,74.122449,12.0,NaN
3,Ba5(InSb3)2,0.0000,eV/atom,NaN,"(Ba, In, Sb)",49.0,56.0,7.0,52.615385,2.603550,...,0.000000,0.000000,0.0,139.0,229.0,90.0,186.076923,33.017751,166.0,NaN
4,NiP2,0.5442,eV/atom,NaN,"(Ni, P)",15.0,28.0,13.0,19.333333,5.777778,...,0.198465,0.264620,0.0,2.0,225.0,223.0,76.333333,99.111111,2.0,NaN


In [16]:
excluded = [ "Property SMILES", "ElementProperty Exceptions"]
excluded_df = computational_df.drop(excluded,axis=1).dropna()
X = excluded_df.drop(["chemical_formula","band_gap","Property band_gap units","composition"],axis=1)
y= excluded_df['band_gap']

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
import numpy as np
min_max_scaler = preprocessing.StandardScaler()
X_train_minmax = min_max_scaler.fit_transform(X)
lr = LinearRegression()

lr.fit(X_train_minmax, y)

# get fit statistics
print('training R2 = ' + str(round(lr.score(X_train_minmax, y), 3)))
print('training RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y, y_pred=lr.predict(X_train_minmax))))

training R2 = 0.605
training RMSE = 1.100


In [144]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=50, random_state=1)

rf.fit(X, y)
print ('R2 = ' + str(round(rf.score(X, y), 3)))
print ('RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y, y_pred=rf.predict(X))))

R2 = 0.963
RMSE = 0.337


In [43]:
experimental_df= pd.read_csv('experimental.csv')
experimental_df.head()

,Chemical formula,Band gap,Crystallinity,Color
0,Bi$_{2}$Te$_{3}$,0.153,Single crystalline,NaN
1,Mg$_{2}$Ge,0.567,Single crystalline,NaN
2,CoSi,0.045,Single crystalline,NaN
3,NaBr,7.025,Single crystalline,NaN
4,Ca$_{2}$Sn,0.9,Polycrystalline,NaN


In [44]:
experimental_df.rename(columns={"Chemical formula":"chemical_formula","Band gap":"band_gap"},inplace=True,errors="raise")

In [45]:
experimental_df['chemical_formula']=experimental_df['chemical_formula'].astype('str') .apply(lambda x:x.translate
                                          ({ord(c): None for c in "_${}"}))
experimental_df['Band gap']=experimental_df['band_gap'].astype('str').apply(lambda t: t[:t.find('$')] if t.find('$')>0 else t).astype(float)

In [46]:
from matminer.featurizers.conversions import StrToComposition
experimental_df = StrToComposition().featurize_dataframe(experimental_df, 'chemical_formula')
from matminer.featurizers.composition import ElementProperty
ep_feat = ElementProperty.from_preset(preset_name="magpie")
experimental_df = ep_feat.featurize_dataframe(experimental_df, col_id="composition")  # input the "composition" column to the featurizer
experimental_df.head()

,chemical_formula,band_gap,Crystallinity,Color,Band gap,composition,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,...,MagpieData range GSmagmom,MagpieData mean GSmagmom,MagpieData avg_dev GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber
0,Bi2Te3,0.153,Single crystalline,NaN,0.153,"(Bi, Te)",52.0,83.0,31.0,64.400000,...,0.000000,0.000000,0.000000,0.0,12.0,152.0,140.0,96.000000,67.200000,152.0
1,Mg2Ge,0.567,Single crystalline,NaN,0.567,"(Mg, Ge)",12.0,32.0,20.0,18.666667,...,0.000000,0.000000,0.000000,0.0,194.0,225.0,31.0,204.333333,13.777778,194.0
2,CoSi,0.045,Single crystalline,NaN,0.045,"(Co, Si)",14.0,27.0,13.0,20.500000,...,1.548471,0.774236,0.774236,0.0,194.0,227.0,33.0,210.500000,16.500000,194.0
3,NaBr,7.025,Single crystalline,NaN,7.025,"(Na, Br)",11.0,35.0,24.0,23.000000,...,0.000000,0.000000,0.000000,0.0,64.0,229.0,165.0,146.500000,82.500000,64.0
4,Ca2Sn,0.9,Polycrystalline,NaN,0.900,"(Ca, Sn)",20.0,50.0,30.0,30.000000,...,0.000000,0.000000,0.000000,0.0,141.0,225.0,84.0,197.000000,37.333333,225.0


In [35]:
experimental_df=experimental_df.dropna(subset=['band_gap'])

In [38]:
excluded = ["band_gap","chemical_formula", "Crystallinity", "Color", "composition"]
X= experimental_df.drop(excluded,axis=1)
y=experimental_df['band_gap'].values

In [39]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

lr = LinearRegression()

lr.fit(X, y)

# get fit statistics
print('training R2 = ' + str(round(lr.score(X, y), 3)))
print('training RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y, y_pred=lr.predict(X))))


ValueError: could not convert string to float: '6.0$\\pm$0.2'